In [1]:
# molecule:Benzocain

In [ ]:
import psi4
print (psi4.__dir__) # this is here so I know if psi4 is importing

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import nglview

In [ ]:
#2D structure generation

benzocain = Chem.MolFromSmiles('O=C(OCC)c1ccc(N)cc1')
benzocain = rdkit.Chem.AddHs(benzocain)
benzocain

In [ ]:
#import experimental structure

AllChem.EmbedMolecule(benzocain)
exp_benzocain = Chem.MolFromMol2File('benzocain.mol2', removeHs=False)
nglview.show_rdkit(exp_benzocain)
#print(exp_benzocain) #just to make sure it's not a none type

In [ ]:
#generate 5 comformers
AllChem.EmbedMultipleConfs(benzocain, 5, randomSeed=1000)

In [ ]:
#calculate RMSD of each of the 5 conformers
#display all 5 generated comformers stacking on experimental structure
#not sure why, but the cell has to be run twice to align the generated and the imported structures

from IPython.display import display

export_data={
    "rmsd_list":[],
    "energy_list":[],
    "opt_energy_list":[],
    "solv_energy_list":[],
    "opt_molecule_list":[]
    }

for ii in range(benzocain.GetNumConformers()):
#for ii in range(1):
    view = nglview.show_rdkit(benzocain, conf_id=ii)
    view.add_component(exp_benzocain)
    
    rmsd = Chem.rdMolAlign.AlignMol(benzocain, exp_benzocain, ii)
    print(f'conformer {ii} has RMSD of {rmsd:0.5f}')
    export_data["rmsd_list"].append(rmsd)
    
    display(view)

In [ ]:
# calculate energy and optimized energy for the comformers generated

psi4.set_memory('5000 MB')
psi4.core.set_output_file('benzocain.txt',False)
#basis = 'B3LYP/6-31G*'
basis = 'HF/3-21G'

#for ii in range(benzocain.GetNumConformers()):
for ii in range(1):
    
    benzocain_geo = psi4.geometry(rdkit.Chem.MolToXYZBlock(benzocain, confId = ii))
    benzocain_energy = psi4.energy(basis, molecule = benzocain_geo)
    #energy_list.append(benzocain_energy)
    print(f'The energy for conformer {ii} is {benzocain_energy:0.5f}')
    
    (opt_energy, opt_wavefunction) = psi4.optimize(basis, molecule = benzocain_geo, return_wfn=True)
    #opt_energy_list.append(opt_energy)
    print(f'conformer {ii} has optimized energy of {opt_energy:0.5f}')
    
    export_data["opt_molecule_list"].append(opt_wavefunction.molecule())
    export_data["energy_list"].append(benzocain_energy)
    export_data["opt_energy_list"].append(opt_energy)

In [ ]:
# pcm string for solvent energy calculation

pcm_string = """
Units = Angstrom
Medium {
  SolverType = IEFPCM
  Solvent = Benzene
}
Cavity {
  RadiiSet = UFF
  Type = GePol
  Scaling = False
  Area = 0.3
  Mode = Implicit
}
"""

psi4.pcm_helper(pcm_string)
psi4.set_options({'pcm':True, 'pcm_scf_type':'total'})

In [ ]:
# energy level in solvent

#for ii in range(benzocain.GetNumConformers()):
for ii in range(1):
    #benzocain_geo = psi4.geometry(rdkit.Chem.MolToXYZBlock(benzocain, confId = ii))
    solv_energy = psi4.energy(basis, molecule = export_data["opt_molecule_list"][ii])
    print(f'conformer {ii} has energy of {solv_energy:0.5f} in benzene')
    
    #solv_energy_list.append(solv_energy)
    
    export_data["solv_energy_list"].append(solv_energy)

In [ ]:
#export the lists into a csv

out_df = pd.DataFrame(export_data)
out_df.to_csv("benzocain_B3LYP631G.csv")

In [ ]:
#sort dataframe to find out the lowest energy of interest

df = pd.read_csv('benzocain_B3LYP631G.csv')
df_opt = df.sort_values(by='opt_energy_list')
print(df_energy.head(1))

df_sol_opt = df.sort_values(by='solv_energy_list')
print(df_sol_opt.head(1))

df_rmsd = df.sort_values(by='rmsd_list')
print(df_rmsd.head(1))

#conformer 1 has the lowest optimized energy, and conformer 0 has the lowest energy in benzene
#neither comforer 1 or 0 has the highest structural agreement with the experimental structure
#conformer 2 has the highest agreement with the experimental structure (smallest RMSD)
#although conformer 2 has the 2nd highest optimized energy, and the highest optimized energy in benzene